In [6]:
import pandas as pd
import json
import torchvision.transforms as transforms
import common
import one_part_dataset

In [7]:
PART_TO_ANALYZE = "Paragolpe Delantero"

metadata = common.load_metadata_dataframe('state.json')
metadata = metadata[metadata.apply(one_part_dataset.is_useful, axis=1)]

complaint_parts = pd.read_csv('../preprocessing/piezas_normalizadas.csv')
display(metadata.head())
display(complaint_parts.head())

,image,useful,angle
0,M204933/M204933001.jpg,yes,frente_acomp
1,M204933/M204933002.jpg,yes,frente_cond
2,M204933/M204933003.jpg,yes,atras_cond
3,M204933/M204933004.jpg,yes,atras_acomp
4,M204933/M204933005.jpg,yes,lado_acomp


,Siniestro,DENUNCIA,Pieza,Tarea,Horas,pieza_normalizada
0,577/2020,M245982,tapa baul hatch,Sustituir,2.50,Tapa de Baul
1,577/2020,M245982,luneta tras,SYC,1.00,Otro
2,577/2020,M245982,kit pegado,Sustituir,0.00,Otro
3,577/2020,M245982,paragolpe trasero,Sustituir,1.00,Paragolpe Trasero
4,577/2020,M245982,farol trasero izq,Sustituir,0.25,Farol Trasero Izquierdo


In [8]:
def parts_info_from_complaint(complaint):
        df_complaint = complaint_parts[complaint_parts["DENUNCIA"] == complaint]
        return list(df_complaint[["pieza_normalizada", "Tarea", "Horas"]].to_records(index=False))

def get_part_category(row):
        is_visible = PART_TO_ANALYZE in common.angulo_pieza[row["angle"]]
        parts_info = parts_info_from_complaint(row["image"].split("/")[0])
        
        broken_part_info = None
        is_broken = False
        for (part, tarea, horas) in parts_info:
            if PART_TO_ANALYZE == part:
                is_broken = True
                broken_part_info = (tarea, horas)

        if is_visible:
            if is_broken:
                return ("roto", broken_part_info[0], float(broken_part_info[1]))
            else:
                return ("sano", "nada", 0)
        else:
            return ("no_visible", "nada", 0)
        
def generate_image_info(row):
    (estado, tarea, horas) = get_part_category(row)
    return (row["image"], estado, tarea, horas)
    

In [9]:
samples = metadata.apply(generate_image_info, axis=1).tolist()
samples_df = pd.DataFrame(samples)
samples_df.columns = ["image", "estado", "tarea", "horas"]

In [10]:
samples_df.to_csv("Distribución Clases {}.csv".format(PART_TO_ANALYZE), index=False)

In [12]:
from IPython.display import display, Markdown, clear_output, Image
import ipywidgets as widgets

select_estado = widgets.Dropdown(options=["roto", "sano", "no_visible"])
select_tarea = widgets.Dropdown(options=[])
select_horas = widgets.Dropdown(options=[])
images = []

def main_function(estado, tarea, horas):
    print (f'{estado}, {tarea}, {horas}')
    global images
    global index
    global max_index
    
    df_filtered_estado = samples_df[samples_df["estado"] == estado]
    df_filtered_tarea = df_filtered_estado[df_filtered_estado["tarea"] == tarea]
    df_filtered_horas = df_filtered_tarea[df_filtered_tarea["horas"] == horas]
    
    images = df_filtered_horas["image"].tolist()
    index = 0
    max_index = len(images) - 1
    
    print (f'Hay {len(images)} de este tipo')

    tarea_options = list(df_filtered_estado["tarea"].unique())
    tarea_options.sort()
    select_tarea.options = tarea_options
    
    horas_options = list(df_filtered_tarea["horas"].unique())
    horas_options.sort()
    select_horas.options = horas_options
    
    with out:
        clear_output()
        print(images[index])
        display(Image(filename='./imgs/' + images[index]))

selects = widgets.interactive(main_function, estado=select_estado, tarea=select_tarea, horas=select_horas)

next_btn = widgets.Button(description='Next')
previous_btn = widgets.Button(description='Previous')
out = widgets.Output()

with out:
    display(Image(filename='./imgs/' + images[index]))

def next_img(_):
    global index
    if index < max_index:
        index += 1
    with out:
      clear_output()
      print(images[index])
      display(Image(filename='./imgs/' + images[index]))

def previous_img(_):
    global index
    if index > 0:
        index -= 1
    with out:
      clear_output()
      print(images[index])
      display(Image(filename='./imgs/' + images[index]))       

# linking button and function together using a button's method
next_btn.on_click(next_img)
previous_btn.on_click(previous_img)

# displaying button and its output together
buttons = widgets.HBox([previous_btn, next_btn])
widgets.VBox([selects, buttons, out])